<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/Province_protezione_civile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

In [12]:
#Carica dati dai file
protezionecivile = pd.read_csv("/content/dpc-covid19-ita-province.csv") 
isLombardia = protezionecivile["denominazione_regione"]=="Lombardia"
protezionecivile = protezionecivile[isLombardia]
notEmpty= protezionecivile["totale_casi"]!= 0
protezionecivile = protezionecivile[notEmpty]
protezionecivile.shape

(2984, 11)